In [ ]:
%reset

In [ ]:
import pandas as pd
import numpy as np

import sys, os

import seaborn as sns
import matplotlib.pyplot as plt

from causalinference import CausalModel

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from importlib import reload

from scipy.stats import wasserstein_distance
from scipy import stats

In [ ]:
def process_data(df, outcome, thresh=0.6):

    # prepare outcomes
    outcomes_to_delete = df.filter(regex='outcome').\
                            columns.\
                            to_list()
    outcomes_to_delete.remove(outcome)
    df.drop(columns=outcomes_to_delete,
            inplace=True)
    df.dropna(subset=[outcome], inplace=True)

    # drop columns with missing values exceeding the thresh
    thresh = round(thresh * len(df.index))
    df = df.dropna(thresh=thresh, axis=1)

    # get dummies
    df = pd.get_dummies(df)
    columns_to_drop = ['gender_M'] + df.filter(regex='False').columns.to_list()
    df.drop(columns=columns_to_drop, inplace=True)

    # convert to bool
    for column in df.select_dtypes(include=['uint8']).columns.to_list():
        df[column] = df[column] == 1

    return df

def get_training_data(df, treatment_col, outcome_col):

    cols_num = df.select_dtypes(include=['float64']).columns.to_list()
    if outcome_col in cols_num:
        cols_num.remove(outcome_col)
    cols_bool = df.select_dtypes(include=['uint8', 'bool']).columns.to_list()
    if treatment_col in cols_bool:
        cols_bool.remove(treatment_col)

    t = df_encoded.loc[:, treatment_col].values
    X_bool = df_encoded[cols_bool].values
    X_num = df_encoded[cols_num].values
    y = df_encoded.loc[:, outcome_col].values


    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_num)
    X_num = imp.transform(X_num)
    scaler = StandardScaler().fit(X_num)
    X_num = scaler.transform(X_num)
    X = np.hstack((X_num, X_bool))

    return y, t, X

def get_covariate_names(df, treatment_col, outcome_col):

    cols_num = df.select_dtypes(include=['float64']).columns.to_list()
    if outcome_col in cols_num:
        cols_num.remove(outcome_col)
    cols_bool = df.select_dtypes(include=['uint8', 'bool']).columns.to_list()
    if treatment_col in cols_bool:
        cols_bool.remove(treatment_col)

    return cols_num + cols_bool

In [ ]:
os.chdir('/home/adam/adam/causal_inference')
from causal_inference.model.propensity_model import PropensityModel
reload(sys.modules['causal_inference.model.propensity_model'])
from causal_inference.model.propensity_model import PropensityModel

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df = pd.read_csv('data_guerin_rct.csv')
df.info(max_cols=200)

In [ ]:
df.loc[df.treated, ['fio2', 'peep', 'po2', 'pf_ratio']].describe()

In [ ]:
df.loc[~df.treated, ['fio2', 'peep', 'po2', 'pf_ratio']].describe()

In [ ]:
OUTCOME = 'pf_ratio_2h_8h_manual_outcome'

In [ ]:
df.loc[df.treated, OUTCOME].describe()

In [ ]:
df.loc[~df.treated, OUTCOME].describe()


In [ ]:
df_encoded = process_data(df=df, outcome=OUTCOME)
df_encoded.info()

y, t, X = get_training_data(df=df_encoded,
                            treatment_col='treated',
                            outcome_col=OUTCOME)

covariates = get_covariate_names(df=df_encoded,
                                 treatment_col='treated',
                                 outcome_col=OUTCOME)

## 3. Causal modelling

#### V1: All variables

In [ ]:
propensity_model_all = PropensityModel(outcome=y,
                                   treatment=t,
                                   covariates=X,
                                   outcome_name=OUTCOME,
                                   treatment_name='proned',
                                   covariates_name=covariates)
propensity_model_all.summary_stats()


In [ ]:
propensity_model_all.est_propensity(X=X, t=t, method='balanced')
propensity_model_all.show_propensity()

In [ ]:
np.mean(abs(propensity_model_all.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_all.trim()

In [ ]:
np.mean(abs(propensity_model_all.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_all.causal_model.blocks = [0, 0.4, 0.6, 0.75, 1]
propensity_model_all.access_balance(method='default')


In [ ]:
propensity_model_all.est_treatment_effect()

In [ ]:
propensity_model_all.print_models(raw_effect=14.4, true_effect=15)

#### With medically relevant features

V2. Balance only potential confounders

In [ ]:
propensity_model_subset = PropensityModel(outcome=y,
                                          treatment=t,
                                          covariates=X,
                                          outcome_name=OUTCOME,
                                          treatment_name='proned',
                                          covariates_name=covariates)
propensity_model_subset.summary_stats()


In [ ]:
propensity_cols = ['pf_ratio',
                   'peep',
                   'fio2',
                   'tidal_volume_per_kg',
                   'nice_copd_True',
                   'nice_hem_malign_True']

In [ ]:
y_subset, t_subset, X_subset = get_training_data(df=df_encoded[propensity_cols],
                                                 treatment_col='treated',
                                                 outcome_col=OUTCOME)

covariates = get_covariate_names(df=df_encoded[propensity_cols],
                                 treatment_col='treated',
                                 outcome_col=OUTCOME)


print(covariates)

In [ ]:
propensity_model_subset.est_propensity(X=X_subset, t=t, method='balanced')
#propensity_model.est_propensity(X=X_subset, t=t_subset, method='balanced')
propensity_model_subset.show_propensity()

In [ ]:
np.mean(abs(propensity_model_subset.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_subset.trim()

In [ ]:
np.mean(abs(propensity_model_subset.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_subset.causal_model.blocks = [0, 0.4, 0.5, 0.6, 0.7, 1]
propensity_model_subset.access_balance(method='default')

In [ ]:
propensity_model_subset.est_treatment_effect()

In [ ]:
propensity_model_subset.print_models(raw_effect=14.3, true_effect=15)

V3. All + interactions + poly

In [ ]:
propensity_model_poly = PropensityModel(outcome=y,
                                          treatment=t,
                                          covariates=X,
                                          outcome_name=OUTCOME,
                                          treatment_name='proned',
                                          covariates_name=covariates)
propensity_model_poly.summary_stats()


In [ ]:
y, t, X = get_training_data(df=df_encoded,
                            treatment_col='treated',
                            outcome_col=OUTCOME)

covariates = get_covariate_names(df=df_encoded,
                                 treatment_col='treated',
                                 outcome_col=OUTCOME)


print(covariates)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
poly.fit_transform(X)
poly = PolynomialFeatures(include_bias=False, interaction_only=True)
X_poly = poly.fit_transform(X)

In [ ]:
propensity_model_poly.est_propensity(X=X_poly, t=t, method='balanced')
#propensity_model.est_propensity(X=X_subset, t=t_subset, method='balanced')
propensity_model_poly.show_propensity()

In [ ]:
np.mean(abs(propensity_model_poly.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_poly.trim()

In [ ]:
np.mean(abs(propensity_model_poly.causal_model.summary_stats['ndiff']))

In [ ]:
propensity_model_poly.causal_model.blocks = [0, 0.4, 0.6, 0.7, 0.8, 1]
propensity_model_poly.access_balance(method='default')

In [ ]:
propensity_model_poly.est_treatment_effect()

In [ ]:
propensity_model_poly.print_models(raw_effect=14.3, true_effect=15)




## Other computations

In [ ]:
n_of_control = []
n_of_treated = []
p_bin = []
effect = []

for stratum in propensity_model_all.causal_model.strata:
    p_min = stratum.summary_stats['p_min'].round(2)
    p_max = stratum.summary_stats['p_max'].round(2)
    index = '[{},{}]'.format(p_min, p_max)
    p_bin.append(index)

    stratum.est_via_matching(bias_adj=True)
    ate = stratum.estimates['matching']['ate']
    effect.append(ate)

    n_of_control.append(stratum.summary_stats['N_c'])
    n_of_treated.append(stratum.summary_stats['N_t'])

df = pd.DataFrame({'n_of_treated': n_of_treated,
                   'n_of_control': n_of_control,
                   'ate': effect}, index=p_bin)

In [ ]:
ax =  df.plot.bar(y=['n_of_control', 'n_of_treated'], ylabel='Frequency', figsize=(10, 5))
df.plot(y='ate', c='k', ax=ax, use_index=False, secondary_y=True, mark_right=False)
ax.right_ax.set_ylabel('ATE')

In [ ]:
# https://seaborn.pydata.org/examples/pairgrid_dotplot.html


In [ ]:
df_balance = pd.DataFrame([], columns=['raw_diff', 'all', 'all_poly', 'subset', 'subset_poly'])
df_balance.loc[:,'raw_diff'] = propensity_model_all.causal_model.summary_stats['ndiff']
df_balance = df_balance.abs().round(2)

In [ ]:
COLUMN_NAME = 'all'
df_balance[COLUMN_NAME] = 0

In [ ]:
df_strat_balance = pd.DataFrame([])
weights = []
n_of_strata = 0

for stratum in propensity_model_all.causal_model.strata:
    n_of_strata += 1
    column = 'strata_{}'.format(n_of_strata)
    df_strat_balance[column] = stratum.summary_stats['ndiff']
    weights.append(stratum.summary_stats['N'])
df_strat_balance = df_strat_balance.abs()


for row, column in df.iterrows():
    print(row)
    df_balance.loc[row, COLUMN_NAME] = np.average(column, weights=np.asarray(weights)).round(2)

In [ ]:
df_balance.loc['mean'] = df.mean()
df_balance.index = propensity_model.covariates_name

In [ ]:
df_balance


In [ ]:
# What if I train using an XGBoost?


def compare_balance()

#### OLS estimation

In [ ]:
import statsmodels.api as sm
import numpy as np

In [ ]:
X.shape

In [ ]:
t.reshape((len(t), 1)).shape

In [ ]:
y.shape

In [ ]:
X_morbid = np.multiply(t, X[:, 14])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
X_new = sm.add_constant(np.hstack((t.reshape((len(t), 1)),X)))
#X_new = np.hstack((X_new, X_morbid.reshape((len(t), 1))))

In [ ]:
poly = PolynomialFeatures(2)
poly.fit_transform(X_new)
poly = PolynomialFeatures(include_bias=True, interaction_only=False)
X_new = poly.fit_transform(X_new)

In [ ]:
poly.get_feature_names()

In [ ]:
X_morbid = X[:, 14]
sum(X_morbid)

In [ ]:
obese = X[:, 14] > 0

In [ ]:
obese

In [ ]:
X_obese = X_new[obese]

X_obese.shape

In [ ]:
sum(t[obese]) / len(t[obese])

In [ ]:
sum(t[~obese]) / len(t[~obese])

In [ ]:
model = sm.OLS(y, X_new)

results = model.fit()

print('R2: ', results.rsquared)
print(results.summary())

In [ ]:
df_ols_summary = pd.DataFrame({'OLS_coef_':results.params.round(2).tolist()},
                              index=['intercept', 'treated'] + covariates)
df_ols_summary['OLS_pvalues_'] = results.pvalues.round(2)
df_ols_summary

In [ ]:
from statsmodels.compat import lzip

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

name = ['Jarque-Bera', 'Chi^2 two-tail prob.', 'Skew', 'Kurtosis']
test = sms.jarque_bera(results.resid)
lzip(name, test)

In [ ]:
# Add interactions

log_reg = sm.Logit(t, X).fit()

In [ ]:
print(log_reg.summary())

In [ ]:
covariates

In [ ]:
import statsmodels.api as sm

log_reg = sm.Logit(t, sm.add_constant(X)).fit()

print(log_reg.summary())

In [ ]:
y, t, X = get_training_data(df=df_encoded[['pf_ratio',
                                          'peep',
                                          'fio2',
                                          'tidal_volume',
                                          'nice_copd_True',
                                          'nice_hem_malign_True']],
                            treatment_col='treated',
                            outcome_col=OUTCOME)

covariates = get_covariate_names(df=df_encoded,
                                 treatment_col='treated',
                                 outcome_col=OUTCOME)

In [ ]:
new_X = np.hstack((X**(i+1) for i in range(k)))

In [ ]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [ ]:
print(X.shape)

print(t.shape)

print(y.shape)

In [ ]:
X_all = np.concatenate((X, t.reshape(len(t), 1)), axis=1)
print(X_all.shape)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y)

In [ ]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse=mean_squared_error(y_test, y_pred, squared=False)

print(rmse)

In [ ]:
from sklearn.metrics import r2_score

coefficient_of_dermination = r2_score(y_test, y_pred)
print(coefficient_of_dermination)

In [ ]:
test_treated = X_test
test_treated[:, -1] = 1
m_1 = xgb_model.predict(test_treated)
test_treated[:, -1] = 0
m_0 = xgb_model.predict(test_treated)

print(np.mean(m_1 - m_0))

In [ ]:
def logit_ip_f(y, X):
    """
    Create the f(y|X) part of IP weights
    from logistic regression

    Parameters
    ----------
    y : Pandas Series
    X : Pandas DataFrame

    Returns
    -------
    Numpy array of IP weights

    """
    model = sm.Logit(y, X)
    res = model.fit()
    weights = np.zeros(X.shape[0])
    weights[y == 1] = res.predict(X[y == 1])
    weights[y == 0] = (1 - res.predict(X[y == 0]))
    return weights

In [ ]:
denoms = logit_ip_f(t, X)
weights = 1 / denoms

In [ ]:
wls = sm.WLS(y, sm.add_constant(X), weights=weights)
res = wls.fit()
print(res.summary())